In [157]:
from collections import defaultdict
import numpy as np
import pickle
import re
from typing import Tuple, DefaultDict, Dict, List, Optional
from tqdm import tqdm

### Zadanie 1. (5p) 
W zadaniu tym masz napisać system, który bierze na wejściu (ztokenizowany) tekst w języku polskim, pozbawiony wielkich liter oraz polskich znaków diakrytycznych i wypisuje na wyjściu poprawny tekst w języku polskim. Zakładamy, że literka „ź” na wejściu jest reprezentowana przez „z” (a nie „x”). Liczymy dwie miary dokładności:

a) Dokładność polskawa, czyli liczba słów poprawnie zrekonstruowanych (modulo wielkość liter, której nie uwzględniamy w tej mierze) podzielona przez liczbę słów w ogóle

b) Dokładność pełna, czyli liczba słów poprawnie zrekonstruowanych podzielona przez liczbę słów (tu uwzględniamy zarówno ogonki jak i wielkość liter).

Ostatecznym wynikiem będzie średnia geometryczna tych liczb. W tym zadaniu sprawdzany jest poziom basic, to znaczy że prezentowane rozwiązanie powinno:
- rekonstruować stokenizowany tekst,
- wykorzystywać dane dotyczące unigramów z części uczącej korpusu,
- w jakiś sposób (dowolny sensowny) uwzględniać informacje o dłuższych ciągach słów.


In [125]:
def preprocess(text: str) -> str:
    text = re.sub('[^a-zA-ZęóąśłżźńĘÓĄŚŁŻŹŃ ]', '', text)
    text = re.sub(' +', ' ', text)
    return text.strip()

In [118]:
def tokenize(text: str) -> str:
    polish_chars_replacements = {'ę': 'e', 'ó': 'o', 'ą': 'a', 'ś': 's', 'ł': 'l', 'ż': 'z', 'ź': 'z', 'ń': 'n'}
    text = text.lower()
    for polish, replacement in polish_chars_replacements.items():
        text = text.replace(polish, replacement)
    return text

In [193]:
def reconstruct_text(
    original: str, 
    corpora_counts: DefaultDict[str, int], 
    bigrams: DefaultDict[str, List[str]],
    bigram_count: DefaultDict[Tuple[str, str], float],
    tokenized_to_word_mapping: Dict[str, str],
) -> str:
    assert len(original) > 0
    reconstructed_words = []
    tokenized = tokenize(original)
    prev_word = None
    for tokenized_word in tokenized.split():
        word = reconstruct_word(prev_word, tokenized_word, corpora_counts, bigrams, bigram_count, tokenized_to_word_mapping)
        reconstructed_words.append(word)
        prev_word = tokenized_word
        
    # Start with a big letter
    first_word_list = list(reconstructed_words[0])
    first_word_list[0] = first_word_list[0].upper()
    reconstructed_words[0] = ''.join(first_word_list)
    return ' '.join(reconstructed_words)

def reconstruct_word(
    prev_word: Optional[str],
    tokenized_word: str, 
    corpora_counts: DefaultDict[str, int], 
    bigrams: DefaultDict[str, List[str]],
    bigram_count: DefaultDict[Tuple[str, str], float],
    tokenized_to_word_mapping: DefaultDict[str, List[str]],
) -> str:
    if tokenized_word not in tokenized_to_word_mapping:
        return tokenized_word
    possible_words = tokenized_to_word_mapping[tokenized_word]
    
    if prev_word is not None and prev_word in bigrams:
        next_words = list(bigrams[prev_word].intersection(set(possible_words)))
        if len(next_words) > 0:
            probs = [bigram_count[prev_word, w] for w in next_words]
            best_word = next_words[np.argmax(probs)]
            return best_word
    
    probs = [corpora_counts[w] for w in possible_words]
    best_word = possible_words[np.argmax(probs)]
    return best_word

In [168]:
def compute_similarity(original: str, reconstucted: str) -> float:
    similarities = [0, 0]
    for original_word, recontructed_word in zip(original.split(), reconstucted.split()):
        if original_word.lower() == recontructed_word.lower():
            similarities[0] += 1
            
            if original_word == recontructed_word:
                similarities[1] += 1
    
    number_of_words = original.count(' ') + 1
    return np.sqrt(similarities[0] * similarities[1]) / number_of_words

In [170]:
corpora_counts = pickle.load(open('data.nogit/corpora_counts.pkl', 'rb'))

In [160]:
saved = pickle.load(open('bigrams_saved.nogit/saved.pkl', 'rb'))

In [164]:
bigrams = saved['from_words']['bigrams']
bigrams_count = saved['from_words']['bigrams_count']

In [120]:
tokenized_to_word_mapping = defaultdict(list)
for word in tqdm(corpora_counts.keys()):
    tokenized_word = tokenize(word)
    tokenized_to_word_mapping[tokenized_word].append(word)

100%|██████████| 2980295/2980295 [00:18<00:00, 160977.29it/s]


In [127]:
# sentences = [
#     'No i to był bardzo ciekawy wyjazd',
#     'Święcie wierzyłem w niezawisłość polskich sądów'
# ]

sentences = open('data.nogit/korpus_prusa.txt', 'r').read().split('\n')

In [195]:
similarities = []
for sentence in tqdm(sentences, position=0, leave=True):
    sentence = preprocess(sentence)
    if len(sentence) == 0:
        continue
    reconstructed_sentence = reconstruct_text(sentence, corpora_counts, bigrams, bigrams_count, tokenized_to_word_mapping)
    similarity = compute_similarity(original=sentence, reconstucted=reconstructed_sentence)
    similarities.append(similarity)
#     print(f'\t\t{similarity}\n{sentence}\n{reconstructed_sentence}\n')
    
print(np.min(similarities), np.mean(similarities), np.max(similarities))

100%|██████████| 16760/16760 [00:14<00:00, 1195.64it/s]

0.0 0.8938724110278521 1.0


### Zadanie 2. (3 + Xp) 
W tym zadaniu rozwiązać należy dokładnie ten sam problem, co w poprzednim zadaniu. Żeby zadanie było uznane za zrobione poprawnie, wynik Twojego programu (na zbiorze ewaluacyjnym), musi być wyższy niż K = 0.955. Dodatkowo, jeżeli wynik R Twojego programu będzie większy niż Y = 0.96, to za zadanie dostaniesz 4 × $\frac{R−Y}{1-Y}$ . Dodatkowa premia to 4 punkty za najlepszy program, 3 punkty za drugie miejsce, 2 punkty za trzecie i 1 punkt za czwarte (liczone we wszystkich grupach). Dozwolone jest korzystanie z korpusu PolEval (pierwszy milion wierszy), N-gramów NKJP oraz Morfologika. Zbiór testowy to kolejne 200 tysięcy wierszy korpusu PolEvala.

In [198]:
def create_validation_set():
    i = 0
    validation_set = []
    with open('data.nogit/polish_corpora.txt', 'r') as f:
        while i < 1e6+200000:
            row = f.readline()
            i += 1
            if i < 1e6:
                continue
            validation_set.append(preprocess(row))
    return validation_set

In [199]:
validation_set = create_validation_set()

In [202]:
similarities = []
for sentence in tqdm(validation_set, position=0, leave=True):
    sentence = preprocess(sentence)
    if len(sentence) == 0:
        continue
    reconstructed_sentence = reconstruct_text(sentence, corpora_counts, bigrams, bigrams_count, tokenized_to_word_mapping)
    similarity = compute_similarity(original=sentence, reconstucted=reconstructed_sentence)
    similarities.append(similarity)
#     print(f'\t\t{similarity}\n{sentence}\n{reconstructed_sentence}\n')
    
print(np.min(similarities), np.mean(similarities), np.max(similarities))

100%|██████████| 200001/200001 [06:41<00:00, 497.59it/s]


0.0 0.8842884585361235 1.0


### Zadanie 3. (4p) 
W zadaniu tym zajmiemy się omawianym na wykładzie ukrytym łańcuchem Markowa, na przykładzie nieuczciwego krupiera rzucającego kością. Przypominam zasady:
1. Krupier ma dwie kości, uczciwą i oszukaną.
2. Kość oszukana daje 6 oczek z p = 1/2, a pozostałe wyniki z p = 1/10
3. Krupier zmienia kość uczciwą na nieuczciwą z p1 = 0.04, a nieuczciwą na uczciwą z p2 = 0.05
4. Zaczynamy od uczciwej kości.

Napisz program, który dla danego ciągu rzutów (który musisz sam wygenerować) wypisuje ciąg stanów (u – kość uczciwa, n – kość nieuczciwa, długość rzędu 10000), w sposób maksymalizujący liczbę prawidłowo zgadniętych stanów. Rozwiąż to zadanie na dwa sposoby:
- Proponując heurystyczny algorytm decydujący na podstawie badania skupisk szóstek
- Implementując poprawny algorytm, bazujący na zmiennych α oraz β (zobacz wykład o HMM).

Wykonując eksperymenty, oszacuj poprawność działania obu algorytmów, mierzoną liczbą poprawnie zgadniętych stanów (podzieloną przez długość ciągu).

### Zadanie 4. (4p) 
W tym zadaniu powinieneś zrekonstruować „parametry” krupiera. Mamy dwie sześcienne kości o nieznanych rozkładach (każdy rozkład to 6 liczb dodatnich, sumujących się do jedynki), zaczynamy od losowo wybranej kości1. Podobnie jak w poprzednim zadaniu p1 i p2 to prawdopodobieństwa zmiany kości. Na SKOSIe znajdziesz zestaw 20000 obserwacji (wyników rzutów kością), poczynionych dla tego modelu (ale do testów możesz też używać danych wygene- rowanych w poprzednim zadaniu). Masz zrekonstruować model, uruchom Twój program dla kilku prefiksów dostępnych danych i porównaj wyniki.

Zastanów się, jak zainicjować model. Czy rozpoczynanie od równych prawdopodobieństw to dobry pomysł?